In [1]:
import warnings
warnings.filterwarnings('ignore')

import copy
import os
import sys
import networkx as nx
import numpy as np
import shutil

# Makes a local copy of current snman code
source_dir = os.path.join('C:',os.sep,'Users','lballo','polybox','GIT','github','snman','snman')
destination_dir = os.path.join('..','snman')
shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

import snman
from snman.constants import *
from snman import osmnx_customized as oxc
from snman import *

#PERIMETER = '_paper2_example'
PERIMETER = '_debug'
#PERIMETER = 'zrh_north-west'

data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [2]:
print('Load street graph')
G = snman.io.load_street_graph(
    os.path.join(outputs_path, 'G_edges.gpkg'),
    os.path.join(outputs_path, 'G_nodes.gpkg'),
    crs=CRS_internal
)

street_graph.organize_edge_directions(G)

Load street graph


In [3]:
from snman import *
import networkx as nx

weight = 'cost_private_cars'
#origin, destination = (1468, 11772)
#origin, destination = (3629, 10509)
origin, destination = (3509, 3513)

H = snman.street_graph.filter_by_hierarchy(G, {snman.hierarchy.MAIN_ROAD})

for lanes_key in [KEY_LANES_DESCRIPTION, KEY_LANES_DESCRIPTION_AFTER]:

    print(lanes_key)
    L = lane_graph.create_lane_graph(H, lanes_attribute=lanes_key)
    route = nx.shortest_path(L, origin, destination, weight=weight)
    
    uvk_and_order = {}
    for u, v in zip(route[:-1], route[1:]):
        k, data = min(L.get_edge_data(u, v).items(), key=lambda e: e[1][weight] + (e[1]['lane'].lanetype == LANETYPE_MOTORIZED))
        uvk_and_order[(u,v,k)] = len(uvk_and_order)
        
    M = L.edge_subgraph(uvk_and_order.keys())
    nx.set_edge_attributes(M, uvk_and_order, 'order_in_path')
    M = lane_graph.LaneGraph(M)
    
    edges = oxc.graph_to_gdfs(M, nodes=False)
    print(edges.groupby('lanetype').agg({weight: 'sum', 'length': 'sum'}))
    print(edges.agg({weight: 'sum', 'length': 'sum'}))
    
    snman.io.export_lane_geometries(
        M,
        os.path.join(outputs_path, f'path_{lanes_key}_edges.gpkg'),
        os.path.join(outputs_path, f'path_{lanes_key}_nodes.gpkg'),
        scaling=30
    )
    

ln_desc
          cost_private_cars       length
lanetype                                
M               6671.538296  6671.538296
cost_private_cars    6671.538296
length               6671.538296
dtype: float64
ln_desc_after
          cost_private_cars        length
lanetype                                 
M              10502.555262  10502.555262
cost_private_cars    10502.555262
length               10502.555262
dtype: float64


In [ ]:
if 0:
    print('export L after')
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)
    snman.io.export_lane_geometries(
        L,
        os.path.join(outputs_path, 'lane_geometries_rebuilt_edges.gpkg'),
        os.path.join(outputs_path, 'lane_geometries_rebuilt_nodes.gpkg')
    )